<a href="https://colab.research.google.com/github/Vinayak-Sharma12/ChatBot_With_History/blob/main/Chatbot_with_ChatHistory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx
!pip install python-pptx
!pip install PyPDF2
!pip install langchain
!pip install langchain_community
!pip install langchain_google_genai
!pip install langchain_text_splitters
!pip install sentence-transformers
!pip install faiss-cpu
!pip install cohere

^C
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/

In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory


In [ ]:
# necessary Imports
from docx import Document
from PyPDF2 import PdfReader
from pptx import Presentation
from langchain_community.llms import Cohere
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts  import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

In [ ]:
#rb means open the file in binary mode which will not give error in text extract
pdf_file = open("/content/RESUME RELATED Q (1).pdf",'rb')
# when you opem a text file 'r' format interprets these bytes according to the system's character encoding (like UTF-8 or ASCII).
#when you open it in rb format it stores data in raw byte format




In [ ]:
# extracting pdf data
pdf_text = ""
pdf_reader = PdfReader(pdf_file)
for page in pdf_reader.pages:
    pdf_text += page.extract_text()

# CHUNKING

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 200, # This is helpul to handle the data loss while chunking.
        length_function = len,
        separators=['\n', '\n\n', ' ', '']
    )

In [ ]:
chunks = text_splitter.split_text(text = pdf_text)

In [ ]:
len(chunks)   #Total number of chunks created after text split

12

# Embedding and Storing it aka INDEXING

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

<ipython-input-8-2f28b5bf73a3>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

In [ ]:
vectorstore = FAISS.from_texts(chunks, embedding = embeddings)

# Retriever

In [ ]:
# creating retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})


In [ ]:
retrieved_docs = retriever.invoke("Companies that come in Thapar?")

In [ ]:
len(retrieved_docs)

5

In [ ]:
print(retrieved_docs[1].page_content)

such
as
representing
my
state
in
Kabaddi
at
the
U-19
level.
Balancing
JEE
preparation,
academics,
and
sports
taught
me
discipline,
resilience,
and
ef fective
time
management.
These
experiences
have
significantly
shaped
my
problem-solving
abilities
and
teamwork,
which
are
invaluable
in
any
professional
setting.
Q2.
What
is
your
jee
percentile
and
your
rank?
Ans2.
->
My
jee
percentile
was
93.43%
and
my
rank
is
68,324
Q3.
What
your
CGP A
is
so
Low?
Ans3.
My
CGP A
reflects
the
balance
I
maintained
between
academic
work
and
hands-on
projects
in
AI/ML,
which
required
significant
time
and
ef fort
outside
of
the
classroom.
During
my
time
at
Thapar
Institute
of
Engineering
and
T echnology ,
I
focused
heavily
on
practical
learning,
internships,
and
personal
projects
like
ThaparGPT
and
the
Plant
Disease
Detection
System.
While
my
academic
grades
are
important,
I
believe
the
real-world
skills
I
developed
in
these
projects—such
as
deep
learning,
NLP ,
and
computer
vision—better
reflect
my


# PROMPTING

In [ ]:
cohere_llm = Cohere(model="command", temperature=0.1, cohere_api_key='sRmFY97EVTJa7VaaaQha5oH7lScl1rxTZv8x6KrV')

<ipython-input-14-20cea1b56291>:1: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  cohere_llm = Cohere(model="command", temperature=0.1, cohere_api_key='sRmFY97EVTJa7VaaaQha5oH7lScl1rxTZv8x6KrV')


In [ ]:
### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    cohere_llm, retriever, contextualize_q_prompt
)

In [ ]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(cohere_llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Elaborate more"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

" During my internship at CSIR, I was involved in developing a solution to address the limitations of LLMs in handling domain-specific queries. My focus was on creating a Cricket-specific LLM that could effectively answer questions related to the sport. To achieve this, I utilized the technique of fine-tuning or domain alignment, which involves training the LLM further on a domain-specific dataset. This approach helped the model learn the terminology, context, and nuances specific to the sport of cricket. \n\nTo enhance the model's responses and address the limitations of LLMs in handling domain-specific queries, I employed the retrieval-augmented generation (RAG) technique. This approach integrates external knowledge sources such as documents, databases, or APIs with the LLM. I used the WikipediaRetriever library from the LangChain community retrievers library to achieve this integration. This enabled the model to fetch information from a domain-specific source to generate more accura

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What are its various way to do"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

In [ ]:
# function to create a single string of relevant documents given by Faiss.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
import os



In [ ]:
# RAG Chain

def generate_answer(question):
    cohere_llm = Cohere(model="command", temperature=0.1, cohere_api_key='sRmFY97EVTJa7VaaaQha5oH7lScl1rxTZv8x6KrV')

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | cohere_llm
        | StrOutputParser()
    )

    return rag_chain.invoke(question)

In [ ]:
ans = generate_answer("Which companies come for placement in Thapar")

In [ ]:
print(ans)

In [ ]:
ans = generate_answer("What is the average package of Thapar Univerisity ")

In [ ]:
question=input('Ask your question')

In [ ]:
ans=generate_answer(question)

In [ ]:
print(ans)